## Unsupervised learning example

In this notebook we explore a classical machine learning approach to a classification problem using a unsupervised learning approach and an algorithm called [K-means clustering](https://en.wikipedia.org/wiki/K-means_clustering). We will use the library [scikit-learn](https://scikit-learn.org/stable/).

In this example we will use the `iris dataset` ([link to the dataset card](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)). 
This dataset is already included in the scikit-learn library, so we do not need to do any preprocessing.

In [ ]:
# Some imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
# Import the dataset
iris = datasets.load_iris()

Let us have a look at the shape of the dataset.

In [ ]:
n_data_points = len(iris['data'])
n_features = len(iris['feature_names'])
feature_names = iris['feature_names']
targets = iris['target_names']
print(f'The dataset has {n_data_points} datapoints')
print(f'The data has {n_features} features, they are {feature_names}')
print(f'The data can have targets {targets}')

Consider only two features (sepal length and sepal width), so that we can plot the datapoints in the plane.

In [ ]:
# Take the first two features. We could avoid this by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

Let's plot the data.

In [ ]:
# Some plotting code (the different colours correspond to different classes)
_, ax = plt.subplots()
X0 = [x[0] for x in X]
X1 = [x[1] for x in X]
ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")

Import the data

Let us divide into training and testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) # You can change the test_size to use more or less data for training

KMeans is an unsupervised learning algorithm, which means that it can be applied to data which is unlabelled. In our case (for the iris dataset) we have labels, and we know that each datapoint belongs to one of three classes. However, in a usual application of unsupervised learning we would not know that, so part of the process is figuring out how many clusters are best for our task. 

The way that KMeans achieves this is as follows. Given a number of clusters $N$ the algorithm tries to divide the dataset in $N$ clusters such that they minimise a quantity called *inertia*. This intuitively measures how cohesive each cluster is.

There are different ways of choosing the number of clusters: one such method is the [ELBOW method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), which finds a good trade-off between number of clusters and inertia. 

Let's fit different KMeans models for different number of clusters and compute their inertia. Then let's plot one against the other to figure out how many clusters we need.

In [ ]:
# Consider a range of possible number of clusters
N_clusters = range(1,21)

# Set somewhere to store the inertia
inertias = []

# Loop over the possible number of clusters and compute the inertia for the corresponding model
for N in N_clusters:
    # Define model
    clf = KMeans(n_clusters=N)

    # Fit the model
    clf.fit(X_train)

    # Compute and store the inertia
    inertias.append(clf.inertia_)

# Plot number of clusters against inertia
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.plot(N_clusters, inertias)

As many things in machine learning this is more of a heuristic than a precise method. In fact, it is hard to see from the graph above why three clusters should be the correct answer! Nevertheless, since we know we have three clusters, let's carry out with $N=3$.

Let us fit the KMeans model.

In [ ]:
clf = KMeans(n_clusters=3)
clf.fit(X_train)

Note that this is not a supervised method, so calculating accuracy makes less sense than when we looked at the SVM. Instead, we plot the areas that are divided into clusters by the KMeans model and compare it to our labels.

In [ ]:
# Just some boring plotting code
_, ax = plt.subplots()
X0, X1 = X[:, 0], X[:, 1]

# Plot the decision boundaries
disp = DecisionBoundaryDisplay.from_estimator(
    clf,
    X_test,
    response_method="predict",
    plot_method="pcolormesh",
    xlabel=iris.feature_names[0],
    ylabel=iris.feature_names[1],
    shading="auto",
    alpha=0.5,
    ax=ax,
)

# Scatter the data
scatter = ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
disp.ax_.legend(
    scatter.legend_elements()[0],
    iris.target_names,
    loc="lower left",
    title="Classes",
)
plt.show()

### Things to try
- What would the final plot look like for different number of clusters?
- There are different metrics for clustering (e.g. silhouette score). Read about them [here](https://scikit-learn.org/1.5/modules/clustering.html#clustering-evaluation) and choose one to evaluate the final model on.
- We have used the first two columns of the dataset. What happens if we use the last two? Or any other two?